In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from scipy.optimize import minimize


X_train = np.load('initial_data/function_8/initial_inputs.npy')
Y_train = np.load('initial_data/function_8/initial_outputs.npy')
#0.231343-0.116511-0.192802-0.000000-1.000000-0.971517-0.109108-0.005687

X_train = np.append(X_train, [[0.231343,0.116511,0.192802,0.000000,1.000000,0.971517,0.109108,0.005687],
                              [0.050917,0.510974,0,0.51982,0.999999,0.964591,0.271744,0.828739],
                              [0.331151,0.212415,0.152484,0,0.999999,0.102819,0.394972,0.789628],
                              [0.,0.,0,0,0.999999,0.734351 ,0.999999,0.195222]], axis=0)
Y_train = np.append(Y_train, [9.6358729591536,9.4260415037804,9.6078496212371,8.512998485868101])

print(X_train)
print(Y_train)

[[0.60499445 0.29221502 0.90845275 0.35550624 0.20166872 0.57533801
  0.31031095 0.73428138]
 [0.17800696 0.56622265 0.99486184 0.21032501 0.32015266 0.70790879
  0.63538449 0.10713163]
 [0.00907698 0.81162615 0.52052036 0.07568668 0.26511183 0.09165169
  0.59241515 0.36732026]
 [0.50602816 0.65373012 0.36341078 0.17798105 0.0937283  0.19742533
  0.7558269  0.29247234]
 [0.35990926 0.24907568 0.49599717 0.70921498 0.11498719 0.28920692
  0.55729515 0.59388173]
 [0.77881834 0.0034195  0.33798313 0.51952778 0.82090699 0.53724669
  0.5513471  0.66003209]
 [0.90864932 0.0622497  0.23825955 0.76660355 0.13233596 0.99024381
  0.68806782 0.74249594]
 [0.58637144 0.88073573 0.74502075 0.54603485 0.00964888 0.74899176
  0.23090707 0.09791562]
 [0.76113733 0.85467239 0.38212433 0.33735198 0.68970832 0.30985305
  0.63137968 0.04195607]
 [0.9849332  0.69950626 0.9988855  0.18014846 0.58014315 0.23108719
  0.49082694 0.31368272]
 [0.11207131 0.43773566 0.59659878 0.59277563 0.22698177 0.41010452
  

In [2]:
# Define Gaussian Process
kernel = RBF(length_scale=np.ones(8)) + WhiteKernel(noise_level=1)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-5, normalize_y=True)
gp.fit(X_train, Y_train.ravel())

C:\Code\RoughWork\Jupyter\venv\lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 5 of parameter k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Code\RoughWork\Jupyter\venv\lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 7 of parameter k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


GaussianProcessRegressor(alpha=1e-05,
                         kernel=RBF(length_scale=[1, 1, 1, 1, 1, 1, 1, 1]) + WhiteKernel(noise_level=1),
                         normalize_y=True)

In [3]:
def ucb(X, gp, kappa=1.96):  # kappa can be adjusted based on your confidence level needs
    mean, std = gp.predict(X, return_std=True)
    return mean + kappa * std


In [4]:
def propose_point(gp, bounds=np.array([[0, 1]]*8), n_restarts=20):
    best_x = None
    best_ucb = float('-inf')

    for starting_point in np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_restarts, 8)):
        res = minimize(lambda x: -ucb(x.reshape(1, -1), gp), x0=starting_point, bounds=bounds, method='L-BFGS-B')
        if -res.fun > best_ucb:
            best_ucb = -res.fun
            best_x = res.x
            
    return best_x

new_point = propose_point(gp)
print("Proposed new hyperparameters:", new_point)

print(np.array2string(new_point,precision=6,separator='-',floatmode='fixed',formatter={'float':'{:0.6f}'.format}))


Proposed new hyperparameters: [0.         0.44547121 0.         0.         0.         0.11118263
 0.         0.92574624]
[0.000000-0.445471-0.000000-0.000000-0.000000-0.111183-0.000000-0.925746]
